<a href="https://colab.research.google.com/github/ealhomsi/ACM/blob/master/ThreeStepSearchPipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install NVIDIA cuda toolkit and other dependencies. We'll run a simple Numba cuda kernel to ensure everything works correctly.


In [0]:
!apt-get install nvidia-cuda-toolkit
!pip3 install numba
!pip3 install opencv-python
!pip3 install wurlitzer
!pip3 install imutils

import os
os.environ['NUMBAPRO_LIBDEVICE'] = "/usr/lib/nvidia-cuda-toolkit/libdevice"
os.environ['NUMBAPRO_NVVM'] = "/usr/lib/x86_64-linux-gnu/libnvvm.so.3.2.0"

from numba import cuda, float32, types
import numba
import numpy as np
import time

@cuda.jit
def hello(data):
    data[cuda.blockIdx.x, cuda.threadIdx.x] = cuda.blockIdx.x * 1024 + cuda.threadIdx.x

numBlocks = 5
threadsPerBlock = 10

data = np.ones((numBlocks, threadsPerBlock), dtype=np.uint8)

hello[numBlocks, threadsPerBlock](data)

print(data)

Reading package lists... Done
Building dependency tree       
Reading state information... Done
nvidia-cuda-toolkit is already the newest version (9.1.85-3ubuntu1).
You might want to run 'apt --fix-broken install' to correct these.
The following packages have unmet dependencies:
 nvidia-cuda-toolkit : Depends: nvidia-cuda-dev (= 9.1.85-3ubuntu1) but it is not going to be installed
E: Unmet dependencies. Try 'apt --fix-broken install' with no packages (or specify a solution).
[[0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]]


We'll define the cuda kernel for performing thhe three step search.

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

DRIVE_PATH = '/content/gdrive/My Drive/BlockMatching/'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import cmath


@cuda.jit(device=True)
def block_is_oob(frame, anchor, block_size):
    return (anchor[0] < 0) or (anchor[1] < 0) or (anchor[0] >= frame.shape[0] - block_size[0]) or (anchor[1] >= frame.shape[1] - block_size[1])
    

@cuda.jit(device=True)
def device_gpu_round(x):
    # Numba bugs out if we use round() built-in in python
    val = float(int(x))
    if abs(x - val) > 0.5:
        return float(int(x + 0.5 * abs(x) / x))
    
    return val


@cuda.jit(device=True)
def search_direction(step_size, z):
    # Use an 8th root of unity to create the rotation towards to the search
    # direction
    sqrt_2 = cmath.sqrt(2)
    scalar = sqrt_2 if z % 2 == 1 else complex(1)

    angle = 2 * 3.14159265359 / 8 * z
    root = cmath.cos(angle) + cmath.sin(angle) * 1j
    rotation = step_size * scalar * root

    # Resolve any truncation errors from using floating points
    # by rounding the real and imaginary parts
    return device_gpu_round(rotation.real) + device_gpu_round(rotation.imag) * 1j


@cuda.jit(device=True)
def device_seq_mae(frame, next_frame, anchor, next_anchor, block_size):
    output = 0

    for i in range(block_size[0]):
        for j in range(block_size[1]):
            error = (
                float(frame[anchor[0] + i, anchor[1] + j]) - 
                float(next_frame[next_anchor[0] + i, next_anchor[1] + j])
            )

            if error < 0:
                error *= -1
            
            output += error
    
    return output / (block_size[0] * block_size[1])


@cuda.jit
def tss_estimation(frame, next_frame, block_size, image_block_layout, motion_vectors):
    errors = cuda.shared.array(shape=(10, 10, 8), dtype=numba.float32)
    anchor = cuda.local.array(shape=(2,), dtype=numba.int32)
    next_anchor = cuda.local.array(shape=(2,), dtype=numba.int32)

    image_block_x = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    image_block_y = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.y

    if image_block_x > image_block_layout[0] or image_block_y > image_block_layout[1]:
        return
    
    if cuda.threadIdx.z == 0:
        motion_vectors[image_block_x, image_block_y, 0] = 0
        motion_vectors[image_block_x, image_block_y, 1] = 0
    
    anchor[0] = image_block_x * block_size[0]
    anchor[1] = image_block_y * block_size[1]

    if device_seq_mae(frame, next_frame, anchor, anchor, block_size) < 5.5:
        return

    step_size = 4

    while step_size >= 1:
        # The direction to search in depends on threadIdx.z
        rotation = search_direction(step_size, cuda.threadIdx.z)
        next_anchor[0] = anchor[0] + rotation.real
        next_anchor[1] = anchor[1] + rotation.imag

        if block_is_oob(frame, next_anchor, block_size):
            errors[cuda.threadIdx.x, cuda.threadIdx.y, cuda.threadIdx.z] = np.inf
        else:
            errors[cuda.threadIdx.x, cuda.threadIdx.y, cuda.threadIdx.z] = device_seq_mae(frame, next_frame, anchor, next_anchor, block_size)

        cuda.syncthreads()

        if cuda.threadIdx.z == 0:
            # Get MSE of current location
            no_motion_mse = device_seq_mae(frame, next_frame, anchor, anchor, block_size)

            argmin_mse = 0
            min_mse = np.inf

            for z in range(8):
                if errors[cuda.threadIdx.x, cuda.threadIdx.y, z] < min_mse:
                    min_mse = errors[cuda.threadIdx.x, cuda.threadIdx.y, z]
                    argmin_mse = z

            if no_motion_mse > min_mse:
                # If the best point is not the center of the search space then we shift the 
                # motion vectors and anchor for the next step by that direction
                direction = search_direction(step_size, argmin_mse)
                motion_vectors[image_block_x, image_block_y, 0] += direction.real
                motion_vectors[image_block_x, image_block_y, 1] += direction.imag

                anchor[0] += direction.real
                anchor[1] += direction.imag
        
        cuda.syncthreads()

        step_size //= 2

Define a function for drawing a motion vector in the image.

In [0]:
def draw_motion_vector(frame, motion_vectors, block):
    # Centroid of the block in pixels
    origin = block_size * block + (block_size / 2)
    point_at = origin + motion_vectors[block[0], block[1]]

    origin = tuple(np.flip(origin.astype(np.int32), axis=0))
    point_at = tuple(np.flip(point_at.astype(np.int32), axis=0))

    cv2.arrowedLine(frame, origin, point_at, (0, 0, 0), 2)


In [0]:
import cv2
import os
import numpy as np


video_shapes = np.array([[360, 640], [720, 1280]])

def find_nearest(frame_shape):
    distances = np.abs(np.sum(video_shapes ** 2 - frame_shape ** 2, axis=1))
    closest = distances.argmin()
    return video_shapes[closest]


def pad_frame(frame, h, w):
    padded_frame = np.zeros((h, w, frame.shape[2]), dtype=np.uint8)
    h_pad = (h - frame.shape[0]) // 2
    w_pad = (w - frame.shape[1]) // 2

    padded_frame[h_pad:h-h_pad, w_pad:w-w_pad, :] = frame
    return padded_frame


def frame_pair_generator(initial_frame, video_capture, read_flag, h, w):
    initial_frame = pad_frame(initial_frame, h, w)

    while read_flag:
        read_flag, frame = video_capture.read()

        if read_flag:
            padded_frame = pad_frame(frame, h, w)
            yield (initial_frame, padded_frame)
            initial_frame = padded_frame
    
    yield (initial_frame, None)


def load_video(video_file):
    video_path = os.path.join(DRIVE_PATH, video_file)
    capture = cv2.VideoCapture(video_path)

    fps = capture.get(cv2.CAP_PROP_FPS)
    frame_count = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
    print("FPS: {0}".format(fps))
    print("Frame count: {0}".format(frame_count))
    
    read_flag, frame = capture.read()
    h, w = find_nearest(np.array(frame.shape[1:3]))

    return frame_pair_generator(frame, capture, read_flag, h, w), fps, frame_count, h, w

We'll now process the video using the three step search algorithm using a pipeline to avoid consuming too much memory.

1. We'll load a pair of frames from the video (x_1, x_2)
2. We'll run three step search on the grayscale images to calculate the motion vectors
3. We'll draw the motion vectors and write the output frame to an output video using ffmpeg to encode the output video.

In [0]:
import tqdm
import subprocess as sp

INPUT_VIDEO = ('baseball_720p', '.mp4')

gen, fps, frames, h, w = load_video(''.join(INPUT_VIDEO))

block_size = np.array([8, 8])
image_block_layout = (h // block_size[0], w // block_size[1])
gpu_block_layout = (10, 10)

# 8 threads per image block, 12*8 image blocks per gpu block
thread_layout = (*gpu_block_layout, 8)

gpu_block_num = tuple((l + (b - 1)) // b for l, b in zip(image_block_layout, gpu_block_layout))

# Initialize some memory for the motion vectors
motion_vectors = np.zeros((*image_block_layout, 2), dtype=np.int8)

cmd = [
    '/usr/bin/ffmpeg',
    '-y',
    '-f', 'image2pipe',
    '-i', '-',
    '-framerate', '%.02f' % fps,
    '-vcodec', 'libx264',
    '-s', '%dx%d' % (w, h),
    '-pix_fmt', 'yuv420p',
    '-preset', 'medium',
    '-an',
    os.path.join(DRIVE_PATH, '%s_out.mp4' % INPUT_VIDEO[0])
]


# Open FFMpeg with a pipe to stdin to pass each frame as a jpeg image and encode an avi video
proc = sp.Popen(cmd, stdin=sp.PIPE, stdout=sp.DEVNULL)

for color_frame, color_next_frame in tqdm.tqdm(gen, 'Processing frames', total=frames):
    # Grayscale transformation
    frame = np.sum(color_frame / color_frame.shape[2], axis=2).astype(np.uint8)

    if color_next_frame is not None:
        next_frame = np.sum(color_next_frame / color_frame.shape[2], axis=2).astype(np.uint8)
        
        # Perform motion estimation on the device with three step search
        tss_estimation[gpu_block_num, gpu_block_layout](frame, next_frame, block_size, np.array(image_block_layout), motion_vectors)

        non_zero_vectors = np.logical_not(np.all(np.equal(motion_vectors, 0), axis=2))
        where = np.where(non_zero_vectors)

        # Draw all non zero motion vectors
        for block_x, block_y in zip(*where):
            draw_motion_vector(color_frame, motion_vectors, np.array([block_x, block_y], dtype=np.int32))

    success, jpeg = cv2.imencode('.jpg', color_frame)
    proc.stdin.write(jpeg.tobytes())


proc.stdin.close()
proc.wait()


Processing frames:   0%|          | 0/1049 [00:00<?, ?it/s]

FPS: 29.97002997002997
Frame count: 1049


Processing frames: 100%|██████████| 1049/1049 [01:29<00:00, 11.72it/s]


0